# 1. Stored Procedure Giriş

* **Stored Procedure konusu SQL Server için çok önemli bir konu ve hem yazılım geliştiriciler hem de veri tabanı yöneticileri çok iyi bir şekilde bu konuyu öğrenmeliler.**

* T-SQL cümlelerinin SQL Server'ın hafızasına kaydedilerek derlendiği ve derlenmiş hallerinin çalıştırıldığı yapılardır.
    * Yani bu T-SQL cümleleri zaten kaydedildiğinde derlenmiş oluyor. Çalıştırıldığı zaman ekstra bir derlenme gerçekleşmiyor. Doğrudan execute işlemi gerçekleştirilerek çalıştırılıyor.

<img src = "https://i.hizliresim.com/jmtvko0.png">

* Bu yöntemde hizmetçiye menemen yaptırmak için tek tek adımların söylenmesine gerek yok. Sadece bir kez menemen yap dediğinde hizmetçi bu adımların hepsini ezberleyecek ve artık her menemen yap deyişte kafasından bu menemeni yapacak.

* SQL Server'a işlemlerin tek tek yaptırılması yerine komple hepsini bir paket olarak yaptırma işlemidir.

### AD HOC QUERY

* Standart SQL veya T-SQL cümleleri ile yapılan tablo veya veritabanı sorgularının her biri **AD HOC QUERY** olarak adlandırılır.
    * VIEWlardan sorgu çekme, fonksiyonlardan sorgu çekme, içeri kayıt atma vs.
    
* Bir AD HOC QUERY çalıştırılırken arka planda 6 farklı adım izlenir.

<img src= "https://i.hizliresim.com/7txfk4z.png" width = 250>

* **Query**: Herhangi bir araç (excel, ssms, bir uygulama) üzerinden sorgunun gönderilmesi.
* **Parse**: Query'nin parse edilmesi. Yani doğruluğunun tespiti. Select yazarken t unutuldu mu gibi. Bu çalıştırılabilir bir sorgu mu? bunun test edilmesi. Query doğruysa bile o tablo veya tablolara erişimin var mı diye yetki kontrolü de yapar.
* **Optimize**: Execution plan'ın çıkarıldığı adımdır. Sorgu çekilirken hangi tablo veya tablolar için hangi indexi kullanmalıyım. İstatistiklere bakarak karar verilir. 
* **Compile**: Execution plan çıkarıldıktan sonra bunun makine diline çevrildiği yani derlendiği aşama. Çalıştırılmaya hazırdır.
* **Execute**: Derlenmiş yapının çalıştırılması. Sonuçları topla.
* **Result**: Sonuçların istenilen ekrana gönderildiği aşama.

* Menemen yapmak için gönderilen 22 adımın her birinde bu işlemler ayrı ayrı izlenir.

### Stored Procedure

* Stored Procedure oluşturulduğunda ve ilk kez çağrıldığında ilk 4 adımın hepsi otomatikmen gerçekleşmiş olur ve makine diline çevrilmiş bir hal alır.

<img src = "https://i.hizliresim.com/8n9fq68.png" width = 250>

* En son halde bu yapı çalıştırılır ve sonuca ulaşılır. 
* Stored Procedure'un performansındaki en önemli etkenlerden biri budur.

# 2. Turnike Örneği Giriş

* Stored Procedure'un daha iyi anlaşılması için bir örnek.

### Stored Procedure- Turnike Kartlı Geçiş Sistemi Örneği
* Bir şirket var ve bu şirkette birçok çalışan var.
* Bu çalışanlar her gün şirkete girebilmek için kartlı geçiş sistemli turnikeleri kullanıyorlar.
* Burada kart okutan insanların çok ciddi bir kontrol mekanizması durumu gerçekleşmektedir.
* Neticede bazı kişiler bu turnikelerden geçebilirken bazıları geçemezler.

<img src = "https://i.hizliresim.com/7dq9i4r.png">

* Şirkette 2000 çalışan varsa ve bu çalışanlar gün içerisinde ortalama 8 kez giriş-çıkış hareketlerinde bulunuyorsa toplamda yıllık olarak yaklaşık 4 milyonluk bir hareket olur.

#### Bir çalışanın giriş işlemi
<img src = "https://i.hizliresim.com/nf4icv5.png">

* Çalışan turnikeye RFID'ye sahip kartını okutuyor.
    * Bu şirkete ait bir kart da olabilir. Akbil gibi, spor salonu giriş kartı gibi okunabilen bir kart da olabilir.
* Daha sonra bu kart tanımlı bir kart mı değil mi diye kontrol edilir. 
    * Eğer tanımlı değilse hata verir, eğer tanımlıysa
* Bu personel dışarıda mı içeride mi diye kontrol edilir.
    * Eğer dışarıda dağilde hata verir, eğer dışarıdaysa
* Girişine izin verilir ve giriş kaydı yapılır.

<img src = "https://i.hizliresim.com/hkvdglp.png">

* Bu işlemler veritabanı üzerinde yukarıdaki gibi gerçekleşir.
* Kart tanımlı mı diye bakılırken **Workers** tablosu üzerinden WHERE şartı ile kaydın olup olmadığı kontrol edilir.
* Daha sonra kayıtlı bulunduğu takdirde **WorkerTransactions** tablosundan tarihe göre ORDER BY yapılıp çalışanın en son tarihli giriş-çıkış işlemi kontrol edilir.
* En son çıkış işlemi yapılmışsa girişe izin verilir ve bu bilgi de **WorkerTransactions** tablosuna giriş kaydı olarak eklenir.

<img src = "https://i.hizliresim.com/kvwo168.png">

* Bu işlemler gerçekleşirken yukarıdaki gibi bir iletişim söz konusudur.
* Çalışan turnikeye kartı okuttuğunda *Turnike Uygulama Sunucusu*, *Veritabanı Sunucu*nun anlayacağı şekilde bir sorgu göndererek ondan tabloda kayıt var mı yok mu diye bir cevap bekler. 
    * Buradaki *Turnike Uygulama Sunucusu* ile *Veritabanı Sunucu* arasındaki işlemler maksimum network hızına bağlıdır. Bu noktada network hızı darboğazdır.
* Cevabı aldıktan sonra; Eğer kayıtlı ise bu sefer;

<img src = "https://i.hizliresim.com/9tphf3f.png">

* En son giriş mi çıkış mı yapılmış diye kontrol edilmesini ister. 
* Eğer en son çıkış yapılmışsa

<img src = "https://i.hizliresim.com/slxyyrd.png">

* Çalışanın girişine izin vererek *WorkerTransactions* tablosuna giriş yapıldığına dair bir kayıt eklenir.

* **Normalde bu kontrollerin daha hızlı yapılması gerekir. Ancak bu işlemlerde sürekli bir git gel söz konusu.**
* **Turnike Uygulama Sunucusu ile Veritabanı Sunucu bir darboğazda kalıyor.**
* **İşlemler Veritabanı Sunucunun raminin hızlılığına hiçbir zaman yetişemeyeceği için bu AD HOC QUERY işlemleri bu noktada zayıf kalıyor.**
* **Peki *Stored Procedure* olursa ne olur?**

# 3. Turnike DB Oluşturma

* Önceki konuda anlatılan turnike örneğinin databaseinin oluşturulması.
* Önce **HR** adında bir database oluşturulur.
* Ardından **WORKERS** ve **WORKERTRANSACTIONS** tabloları oluşturulur.
    * WORKERS tablosundaki WORKERBARCODE kolonu benzersiz olması için *uniqueidentifier* veri türüyle tanımlanmıştır.
* **WORKER** tablosuna veri aşağıdaki gibi atılır; 

<img src = "https://i.hizliresim.com/3fcu1gg.png">

* Tüm kolonları almak için *Columns*ı sürükleyebilirsin.
* <code>NEWID()</code> rastgele bir değer üretir.

<img src = "https://i.hizliresim.com/fgtn2mk.png">

* Yukarıdaki şekilde bir personelin bir günlük şirkete giriş-çıkışları veritabanına kayıt edilmiştir.
* Bu sadece bir personel içindir. Bu işlemleri bütün personeller için düşündüğümüzde bu yapı büyümektedir.
* Bu durumun kontrol edilmesi ve performansı kontrol edilmesi gerekmektedir.

# 4. Turnike Similasyonu Ad Hoc Query
* Şu anda **WORKERTRANSACTIONS** tablomuza üst üste aynı kullanıcıdan *G* için veri kayıt edebiliriz. Ancak normalde bu kontrolün Uygulama Sunucusu tarafından sağlanması gerekiyor.
* Bu yüzden similasyon uygulaması kullanıldı.
* Similasyon uygulamasında sistem yukarıda anlatıldığı gibi işliyor.

<img src = "https://i.hizliresim.com/55tpsiq.png">

# 5. İlk Stored Procedure

* Önceki derste AD HOC QUERY ile yapılan turnike işlemleri STORED PROCEDURE ile denenecek ancak önce STORED PROCEDURE'ün nasıl tanımlandığı ve çalıştığı görünmesi gerekiyor. O yüzden aşama aşama gidilecek.

### 1. Kullanıcının kartının sisteme kayıtlı olup olmadığını bulan stored procedure
* Stored procedureler içerisinde T-SQL komutları barındıran komut topluluklarıdır.
* Aynı fonksiyon, tablo ve viewlarda olduğu gibi CREATE, ALTER ve DROP komutlarıyla işlemleri gerçekleştirilir.
* Procedureler de fonksiyonlar gibi parametreler alır ve parametrelere göre çağrılır.
* Stored procedure tanımlarken anahtar kelime **PROCEDURE**dür ve ardından procedure ismi yazılır.
    * Genellikle bu veritabanı nesnesinin stored procedure olduğunu belirtmek adına nesne ismi *SP* ile başlar.
* Parametreler fonksiyonlarda olduğu gibi parantez içerisinde de tanımlanabilirken fonksiyonlarda olmayan şekliyle parantez olmadan da tanımlanabilir. Ve eğer birden fazla parametre varsa yine virgül ile ayrılır.
* **AS** diyerek devam edilir ve **BEGIN-END** bloğu açılarak procedure içerisinde gerekli işlemler yapılır.

<img src = "https://i.hizliresim.com/jzx6221.png">

* Oluşturulan bu procedureler veritabanı içerisinde *Programmability*'nin altında *Stored Procedures* klasöründe tutulur.
* Oluşturulan procedure ilgili parametre veya parametreleriyle beraber **EXEC** komutu ile çağrılır. Eğer parametre girilmezse hata verir.

<img src = "https://i.hizliresim.com/axi9m3t.png" >

* Bir stored procedure'e parametre iki şekilde gönderilir. Ya parametreleri olduğu gibi sırasına göre virgülle ayırarak yazarsın. 

<img src = "https://i.hizliresim.com/amaky2h.png">

* Ya da parametreleri sıradan bağımsız olarak parametre ismine göre belirterek yazarsın.

<img src = "https://i.hizliresim.com/3n14id3.png">

* Procedure'ün şu anki haliyle parametre gönderildiğinde kart bilgisi WORKERS tablosuyla eşleşir ise çalışanın bilgileri döndürülür.

<img src = "https://i.hizliresim.com/638jwel.png">

* Eşleşmez ise şu anda hiçbir değer gelmez.

<img src = "https://i.hizliresim.com/7c6ip7n.png">

* Şu anda bu aşamada bu Stored Procedure'ün AD HOC QUERY'den pek farkı yoktur.
* Farkı açmak adına diyelim ki eğer girilen kart geçerli değilse ekrana hata bassın.

<img src = "https://i.hizliresim.com/rrvzeyc.png">

* Bu noktada procedure üzerinde değişiklik yapacağımız için **ALTER** komutunu kullanırız.
* **IF EXISTS** komutu kayıt döndürürse anlamı taşır. Buna göre işlemler yapılır ve döndürmezse diye de **ELSE** konur.

* Bu noktada doğru veya yanlış kart numaraları girildiği takdirde, yani geçerli veya geçersiz kartlar turnikeye okutulduğu takdirde aşağıdaki çıktılar alınır.

<img src = "https://i.hizliresim.com/3itnliw.png">

* Procedure içerisinde sadece **IF** şartları çalışmaz. **DECLARE** ile değişken de tanımlanabilir.

<img src = "https://i.hizliresim.com/1p4fd9b.png">

* Güncellenen procedure'de *WORKERNAME* isimli değişken procedure içerisinde tanımlanır.
* Bu değişkene WHERE şartı ile değer ataması yapılır. Değer ataması parametrenin WORKERS tablosunda bir karşılığı var ise gerçekleşir. Yani kart sahibi bulunursa değişkene ismi atanır bulunamazsa değişken NULL döndürür.
* Bu yüzden IF-ELSE komutunda *WORKERNAME* **IS NULL** ise kartın geçersiz olduğu belirtilir. **ELSE** kart sahibinin ismi döndürülür.

<img src = "https://i.hizliresim.com/mr3ma1a.png">

* Bu noktada kartın geçersiz olması bir hata belirttiğinden dolayı bunu biz de SQL Server'da hata mesajı olarak bastırabiliriz.
* Bunu sağlayan komut **RAISERROR**dür. Ve hatanın devamında procedureun çalışmasına gerek olmadığı için **RETURN** ile çalışmayı durdururuz.

<img src = "https://i.hizliresim.com/rtnni7a.png">

<img src = "https://i.hizliresim.com/tk2c6lc.png">

* Bu şekilde birden çok adım toplu şekilde procedureler içerisinde yazılabilir.

# 6. Turnike Giriş Çıkış Procedure

* Kart kontrolü yapılır.
    * Kart yanlışsa hata mesajı bastırılır.
    * Kart doğruysa;
* İşlem yapan kişinin son etkinliği kontrol edilir.
    * Son işlem ile yapılmak istenen işlem aynıysa hata mesajı bastırılır.
    * Aynı değilse;
* İşlem veritabanına kayıt edilir ve ekrana bastırılır.

<img src = "https://i.hizliresim.com/abt4sb3.png">

* İşlemlerin tümü procedure üzerinden gerçekleştirilir. Diyelim kart geçersiz.

<img src = "https://i.hizliresim.com/48eq4ia.png">

* Database bazında kontrol sonlanır, diğer işlemlere gerek duyulmaz.
* Bütün işlemler client-server mimarisine ihtiyaç duyulmadan stored procedure içerisinde gerçekleşir.
* Bu artık network hızında değil ram hızında çalışıyorum demektir.
* Ayrıca her kontrol sorgusu için 6 adım yerine doğrudan execute edilir.

----
* Parametreleri karışık sıra ile verirsen hata alırsın.

<img src = "https://i.hizliresim.com/ri8zmcb.png">

* Ancak parametre isimlerini belirtirsen sıkıntı olmaz.

----

* Arkadaşınla molaya çıkacaksınız, arkadaşının masası çok dağınıktı kartını bulamadı, gel şimdi benimkini okutursun sonra bulursun seninkini dedin, sen turnikeden 10:05:20 de geçtin 5 saniye sonra o da çıksın diye kartı ona verdin, o da kartını okuttu;

<img src = "https://i.hizliresim.com/ebbilnc.png"> 

* Giriş için de benzer hatayı alırsın.

* İşlemler tutarlı şekilde devam ettiğinde bir kişinin günlük giriş çıkışları **tek bir procedure** üzerinden gerçekleştirilebilir ve bu da aşağıdaki gibi görünür.

<img src = "https://i.hizliresim.com/ckgrdt9.png">

# 7. Turnike Similasyonu Stored Procedure

